# BA 476 Team 10 Jupyter Notebook

In [ ]:
%%html
<style>
.cell-output-ipywidget-background {
    background-color: transparent !important;
}
:root {
    --jp-widgets-color: var(--vscode-editor-foreground);
    --jp-widgets-font-size: var(--vscode-editor-font-size);
}  
</style>

## Setup

In [ ]:
from pathlib import Path

import kagglehub
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, StackingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, root_mean_squared_error
from sklearn.model_selection import GridSearchCV, KFold, train_test_split
from sklearn.preprocessing import StandardScaler
from tqdm.auto import tqdm
from xgboost import XGBRegressor, XGBRFRegressor

In [ ]:
# To collect results from all models as we go
full_results = pd.DataFrame()

# Color palette
sns.set(
    rc={
        "axes.facecolor": (1, 1, 1, 0),
        "figure.facecolor": (1, 1, 1, 0),
        "text.color": "lightgray",
        "xtick.color": "lightgray",
        "ytick.color": "lightgray",
    }
)
spotify_palette = sns.diverging_palette(279, 141, s=92, l=68, center="light", as_cmap=True)
spotify_colors = sns.diverging_palette(279, 141, s=92, l=68, center="light")

## Data Download and Processing
Download the supplementary data from Kaggle for artist info

In [ ]:
if Path("data/spotify_artist_data.csv").exists():
    artist_stats = pd.read_csv("data/spotify_artist_data.csv")
else:
    path = kagglehub.dataset_download("adnananam/spotify-artist-stats")
    artist_stats = pd.read_csv(path + "/spotify_artist_data.csv", index_col=0)

    # Remove error rows b/c the creator didn't process correctly
    artist_stats = artist_stats[artist_stats["Lead Streams"] != "Lead Streams"]

    # Cast numeric columns to int
    for col in ["Lead Streams", "Feats", "Tracks", "One Billion", "100 Million"]:
        artist_stats[col] = artist_stats[col].str.replace(",", "").astype(int)

    # Remove the last updated column, it's not useful/relevant
    artist_stats = artist_stats.drop(columns=["Last Updated"])

    artist_stats.to_csv("data/spotify_artist_data.csv", index=False)

artist_stats.head()

Download the dataset from HuggingFace using Pandas, and drop the extra index column. The `na`/`NaN` values were dropped from the `artists` column because that column is used to merge the supplementary data above with the main dataset.

In [ ]:
# Pulled dataset from HF, dropped unneeded index column
if Path("data/spotify_tracks.csv").exists():
    df = pd.read_csv("data/spotify_tracks.csv")
else:
    df = (
        pd.read_csv("hf://datasets/maharshipandya/spotify-tracks-dataset/dataset.csv")
        .drop("Unnamed: 0", axis=1)
        .dropna(subset=["artists"])
    )

    df["duration_s"] = df["duration_ms"] / 1000
    df = df.drop(columns=["duration_ms"])  # Drop original duration column, keep seconds

    df.to_csv("data/spotify_tracks.csv", index=False)

df_nodupe = df.drop_duplicates(subset=["track_id"]).copy()

df.head()

Adding in more information to the main dataset using each artist's stats. If there are two or more artists present, the stats are averaged.

Stats merged:

- Lead streams
- Streams of features
- Number of tracks
- Number of songs with more than one billion streams
- Number of songs with more than 100 million streams

The second half of the cell creates dummy variables for the `genres` column. The genre column and duplicate song entries are then dropped from the dataframe. Each song is repeated $x$ number of times where $x$ is the number of genres it has.

In [ ]:
if not Path("data/spotify_tracks_processed.csv").exists():
    # Adding in information based on the artist stats (merge on names)
    art_stats_name = set(artist_stats["Artist Name"].values)
    lead_streams, feats, tracks, one_billion, hundred_million = [], [], [], [], []

    for row in tqdm(df_nodupe.iterrows(), total=df_nodupe.shape[0], desc="Processing rows"):
        artists = [x.strip() for x in row[1]["artists"].split(";")]
        temp_lead_streams, temp_feats, temp_tracks, temp_one_billion, temp_hundred_million = (
            [],
            [],
            [],
            [],
            [],
        )

        for artist in artists:
            if artist in art_stats_name:
                temp_lead_streams.append(
                    artist_stats[artist_stats["Artist Name"] == artist]["Lead Streams"].values[0]
                )
                temp_feats.append(artist_stats[artist_stats["Artist Name"] == artist]["Feats"].values[0])
                temp_tracks.append(
                    artist_stats[artist_stats["Artist Name"] == artist]["Tracks"].values[0]
                )
                temp_one_billion.append(
                    artist_stats[artist_stats["Artist Name"] == artist]["One Billion"].values[0]
                )
                temp_hundred_million.append(
                    artist_stats[artist_stats["Artist Name"] == artist]["100 Million"].values[0]
                )

        for col, temp in zip(
            [lead_streams, feats, tracks, one_billion, hundred_million],
            [temp_lead_streams, temp_feats, temp_tracks, temp_one_billion, temp_hundred_million],
            strict=True,
        ):
            if len(temp) > 0:
                col.append(np.mean(temp))
            else:
                col.append(0)

    df_nodupe["lead_streams"] = lead_streams
    df_nodupe["feats"] = feats
    df_nodupe["tracks"] = tracks
    df_nodupe["one_billion"] = one_billion
    df_nodupe["hundred_million"] = hundred_million


    # Creating dummy variables based on genres
    g_dummy = pd.get_dummies(df["track_genre"]).groupby(df["track_id"]).sum().astype(int).reset_index()

    dummy_val = g_dummy.copy()
    dummy_val["total"] = dummy_val.sum(axis=1, numeric_only=True)
    dummy_val = dummy_val[["track_id", "total"]].sort_values("track_id", ascending=True)

    process_check = (
        df.groupby("track_id")
        .size()
        .to_frame("total")
        .reset_index()
        .sort_values("track_id", ascending=True)
    )

    for df1, df2 in zip(process_check.iterrows(), dummy_val.iterrows(), strict=True):
        assert (df1[1]["total"] == df2[1]["total"]) and (df1[1]["track_id"] == df2[1]["track_id"])

    df = df_nodupe.merge(g_dummy, on="track_id").drop(
        ["track_id", "artists", "album_name", "track_name", "track_genre"], axis=1
    )
    df["explicit"] = df["explicit"].astype(int)
    df.to_csv("data/spotify_tracks_processed.csv", index=False)

else:
    df = pd.read_csv("data/spotify_tracks_processed.csv")

### Baseline Linear Regression Model

A quick test of the linear regression model using only the base data and dummy variables made from genres.

In [ ]:
# Assuming 'df' is your DataFrame and 'features' is a list of feature column names
X = df[
    df.columns.difference(
        ["popularity", "lead_streams", "feats", "tracks", "one_billion", "hundred_million"]
    )
]
y = df["popularity"]

# Split the dataset into train and test sets (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

# Initialize the LinearRegression model
model = LinearRegression(
    n_jobs=-1,
)

# Fit the model to the training data
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate R² and MSE
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = root_mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

full_results = pd.concat(
    [
        full_results,
        pd.DataFrame(
            {
                "Model": "Baseline Linear Regression",
                "Dataset": ["Test", "Train"],
                "R²": [r2, r2_score(y_train, model.predict(X_train))],
                "MSE": [mse, mean_squared_error(y_train, model.predict(X_train))],
                "RMSE": [rmse, root_mean_squared_error(y_train, model.predict(X_train))],
                "MAE": [mae, mean_absolute_error(y_train, model.predict(X_train))],
            }
        ),
    ]
)

# Output the results
print(f"R²: {r2}")
print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"MAE: {mae}")

## Additional Data Processing

### Backfill missing `lead_streams` values

Fill in values for `lead_streams` using all columns except for `lead_streams` and `popularity`.

In [ ]:
# Create mask for rows where lead_streams is 0
mask = df['lead_streams'] == 0

# Split data into features (X) and target (y)
X_train = df[~mask].drop(['lead_streams', 'popularity'], axis=1)
y_train = df[~mask]['lead_streams']

# Prepare features for prediction
X_pred = df[mask].drop(['lead_streams', 'popularity'], axis=1)

# Initialize and train the RandomForestRegressor
rf_model = RandomForestRegressor(
    n_estimators=200, 
    random_state=42, 
    n_jobs=-1, 
    max_features='sqrt',
    verbose=1
)
rf_model.fit(X_train, y_train)

# Make predictions for empty values
predictions = rf_model.predict(X_pred)

# Fill in the empty values
df.loc[mask, 'lead_streams'] = predictions

# Verify no more zeros in lead_streams
print(f"Number of zeros in lead_streams: {(df['lead_streams'] == 0).sum()}")

### Add clusters as additional features

KMeans clusters as an additonal feature for the models to use.

In [ ]:
std_df = StandardScaler().fit_transform(df[df.columns.difference(["popularity"])])
kmeans = KMeans(n_clusters=40, random_state=42)
kmeans.fit(std_df)
df["cluster"] = kmeans.labels_
df["cluster"] = df["cluster"].astype("category")

### Manual correlation check

In [ ]:
dfc = df.corr()

# Create mask for correlations > abs(0.50)
mask = np.abs(dfc) > 0.50

# Get upper triangle of mask to avoid duplicates
mask_upper = np.triu(mask, k=1)

# Find correlation pairs exceeding threshold
high_corr = []
for i in range(len(dfc.columns)):
    for j in range(i + 1, len(dfc.columns)):
        if mask_upper[i, j]:
            high_corr.append({"var1": dfc.columns[i], "var2": dfc.columns[j], "corr": dfc.iloc[i, j]})

# Convert to dataframe and sort by absolute correlation
high_corr_df = pd.DataFrame(high_corr)
high_corr_df = high_corr_df.sort_values("corr", key=abs, ascending=False)

print("Correlations > |0.50|:")
print(high_corr_df.to_string(index=False))

### Drop highly correlated columns

- `singer-songwriter`
  - Removed since it is an identical match to `songwriter`

In [ ]:
df = df.drop(columns=["singer-songwriter"])

## Models and Evaluation

### Baseline Linear Regression

This is another run of the Linear Regression Model but with using the data with extra features.

In [ ]:
# Assuming 'df' is your DataFrame and 'features' is a list of feature column names
X = df[df.columns.difference(["popularity"])]
y = df["popularity"]

# Split the dataset into train and test sets (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

# Initialize the RandomForestRegressor model
model = LinearRegression(
    n_jobs=-1,
)

# Fit the model to the training data
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate R² and MSE
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = root_mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

full_results = pd.concat(
    [
        full_results,
        pd.DataFrame(
            {
                "Model": "Post-processing Linear Regression",
                "Dataset": ["Test", "Train"],
                "R²": [r2, r2_score(y_train, model.predict(X_train))],
                "MSE": [mse, mean_squared_error(y_train, model.predict(X_train))],
                "RMSE": [rmse, root_mean_squared_error(y_train, model.predict(X_train))],
                "MAE": [mae, mean_absolute_error(y_train, model.predict(X_train))],
            }
        ),
    ]
)

# Output the results
print(f"R²: {r2:.4f}")
print(f"MSE: {mse:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")

### Random Forest Regressor

In [ ]:
# Assuming 'df' is your DataFrame and 'features' is a list of feature column names
X = df[df.columns.difference(["popularity"])]
y = df["popularity"]

# Split the dataset into train and test sets (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Initialize the RandomForestRegressor model
model = RandomForestRegressor(
    n_estimators=200, random_state=42, n_jobs=-1, max_features="sqrt", bootstrap=True
)

# Fit the model to the training data
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate R² and MSE
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = root_mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

full_results = pd.concat(
    [
        full_results,
        pd.DataFrame(
            {
                "Model": "Random Forest Regressor",
                "Dataset": ["Test", "Train"],
                "R²": [r2, r2_score(y_train, model.predict(X_train))],
                "MSE": [mse, mean_squared_error(y_train, model.predict(X_train))],
                "RMSE": [rmse, root_mean_squared_error(y_train, model.predict(X_train))],
                "MAE": [mae, mean_absolute_error(y_train, model.predict(X_train))],
            }
        ),
    ]
)

# Output the results
print(f"R²: {r2:.4f}")
print(f"MSE: {mse:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")

### XGBoost Regressor

Given this is a boosting model, `early_stopping_rounds` has been set to 50 to avoid overfitting on the train and validation data. But unlike the other models, this one gets a custom `75:15:10` train/validation/test split.

In [ ]:
# Assuming 'df' is your DataFrame and 'features' is a list of feature column names
X = df[df.columns.difference(["popularity"])]
y = df["popularity"]

# Split the dataset into train, test, and validation sets (75% train, 15% test, 10% validation)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.25, random_state=42)

X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=40, random_state=42)

# Initialize the XGBRegressor model
model = XGBRegressor(
    tree_method="hist",
    n_estimators=300,
    n_jobs=-1,
    random_state=42,
    enable_categorical=True,
    early_stopping_rounds=50,
)

# Fit the model to the training data
model.fit(X_train, y_train, eval_set=[(X_val, y_val)])

# Make predictions on the test set
y_pred = model.predict(X_test)

In [ ]:
# Calculate R² and MSE
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = root_mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

full_results = pd.concat(
    [
        full_results,
        pd.DataFrame(
            {
                "Model": "XGBoost Regressor",
                "Dataset": ["Test", "Train"],
                "R²": [r2, r2_score(y_train, model.predict(X_train))],
                "MSE": [mse, mean_squared_error(y_train, model.predict(X_train))],
                "RMSE": [rmse, root_mean_squared_error(y_train, model.predict(X_train))],
                "MAE": [mae, mean_absolute_error(y_train, model.predict(X_train))],
            }
        ),
    ]
)


# Output the results
print(f"R²: {r2:.4f}")
print(f"MSE: {mse:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")

### XGBoost Random Forest Regressor

In [ ]:
# Assuming 'df' is your DataFrame and 'features' is a list of feature column names
X = df[df.columns.difference(["popularity"])]
y = df["popularity"]

# Split the dataset into train and test sets (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

# Initialize the XGBRFRegressor model
model = XGBRFRegressor(
    tree_method="hist", n_estimators=200, n_jobs=-1, random_state=42, enable_categorical=True
)

# Fit the model to the training data
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate R² and MSE
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = root_mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

full_results = pd.concat(
    [
        full_results,
        pd.DataFrame(
            {
                "Model": "XGBoost Random Forest Regressor",
                "Dataset": ["Test", "Train"],
                "R²": [r2, r2_score(y_train, model.predict(X_train))],
                "MSE": [mse, mean_squared_error(y_train, model.predict(X_train))],
                "RMSE": [rmse, root_mean_squared_error(y_train, model.predict(X_train))],
                "MAE": [mae, mean_absolute_error(y_train, model.predict(X_train))],
            }
        ),
    ]
)

# Output the results
print(f"R²: {r2:.4f}")
print(f"MSE: {mse:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")

## Plots

#### Plot of RMSE and MAE for each model

In [ ]:
# Set figure size
plt.figure(figsize=(12, 6))

plt_df = full_results[full_results["Dataset"] == "Test"].sort_values("MSE", ascending=False)

# Create bar positions
x = range(len(plt_df))

# Create grouped bars for each metric with adjusted positions
bar_width = 0.2
bars1 = plt.bar(
    [i - bar_width / 2 for i in x],
    plt_df["RMSE"],
    bar_width,
    label="RMSE",
    color=spotify_colors[-1],
    edgecolor="black",
    linewidth=0.5,
)
bars2 = plt.bar(
    [i + bar_width / 2 for i in x],
    plt_df["MAE"],
    bar_width,
    label="MAE",
    color=spotify_colors[0],
    edgecolor="black",
    linewidth=0.5,
)

# Add value labels on top of each bar
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        plt.text(
            bar.get_x() + bar.get_width() / 2.0,
            height + 0.3,
            f"{height:.2f}",
            ha="center",
            va="bottom",
            fontsize=9,
        )

# Customize the plot
plt.xlabel("Models", color="lightgray", fontsize=12)
plt.ylabel("Score", color="lightgray", fontsize=12)
plt.title("Model Performance Comparison - RMSE and MAE", fontsize=14)

# Position x-ticks at the center of each model's bar group
# Format labels with newlines between words
labels = plt_df["Model"].str.replace(" ", "\n", regex=False)
plt.xticks(x, labels, rotation=0, ha="center", fontsize=10)

# Improve legend readability
plt.legend(loc="upper right", fontsize=12, frameon=True, facecolor="black", edgecolor="gray")
plt.grid(True, axis="y", linestyle="--", alpha=0.7)
plt.grid(False, axis="x")  # Disable grid for x-axis
plt.ylim(0, 20)  # Set y-axis limit

# Adjust layout to prevent label cutoff
plt.tight_layout()

plt.savefig("images/rmse-mae.png", dpi=300, bbox_inches="tight", transparent=True)

plt.show()

#### Plot of MSE for each model

In [ ]:
# Set figure size
plt.figure(figsize=(12, 6))

plt_df = full_results[full_results["Dataset"] == "Test"].sort_values("MSE", ascending=False)

# Create bar positions
x = range(len(plt_df))

# Create bar plot for MSE
bars = plt.bar(x, plt_df["MSE"], width=0.5, color=spotify_colors[-1], label="MSE")

# Add value labels on top of each bar
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2.0, height, f"{height:.2f}", ha="center", va="bottom")

# Customize the plot
plt.xlabel("Models", color="lightgray", fontsize=12)
plt.ylabel("Score", color="lightgray", fontsize=12)
plt.title("Model Performance Comparison - MSE")

# Position x-ticks at the center of each model's bar group
# Format labels with newlines between words
labels = plt_df["Model"].str.replace(" ", "\n", regex=False)
plt.xticks(x, labels, rotation=0, ha="center", fontsize=10)

plt.legend(loc="upper right", fontsize=12, frameon=True, facecolor="black", edgecolor="gray")
plt.grid(True, axis="y", linestyle="--", alpha=0.7)
plt.grid(False, axis="x")  # Disable grid for x-axis

plt.ylim(0, 350)  # Set y-axis limit to 10% above max value

# Adjust layout to prevent label cutoff
plt.tight_layout()

plt.savefig("images/mse.png", dpi=300, bbox_inches="tight", transparent=True)

plt.show()

#### Predictions vs Actual

In [ ]:
# Get unique models from the results
models = full_results[full_results['Dataset'] == 'Test']['Model'].unique()

# Prepare X and y data
X = df[df.columns.difference(["popularity"])]
y = df["popularity"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

# Dictionary to store model instances
model_instances = {
    "Baseline Linear Regression": LinearRegression(n_jobs=-1),
    "Post-processing Linear Regression": LinearRegression(n_jobs=-1),
    "Random Forest Regressor": RandomForestRegressor(
        n_estimators=200, random_state=42, n_jobs=-1, max_features="sqrt"),
    "XGBoost Regressor": XGBRegressor(
        tree_method="hist", n_estimators=300, n_jobs=-1, random_state=42, 
        enable_categorical=True),
    "XGBoost Random Forest Regressor": XGBRFRegressor(
        tree_method="hist", n_estimators=200, n_jobs=-1, random_state=42, 
        enable_categorical=True)
}

# Create separate figures for each model
for i, model_name in enumerate(models):
    # Create a new figure for each model
    plt.figure(figsize=(8, 7))
    
    # Train the model
    model = model_instances[model_name]
    model.fit(X_train, y_train)
    
    # Make predictions
    y_pred = model.predict(X_test)
    
    # Create scatter plot of actual vs predicted values
    plt.scatter(y_test, y_pred, alpha=0.3, color=spotify_colors[i % len(spotify_colors)])
    
    # Add perfect prediction line
    min_val = 0
    max_val = 100
    plt.plot([min_val, max_val], [min_val, max_val], '--', lw=2, color='cyan')
    
    # Calculate metrics for the title
    r2 = r2_score(y_test, y_pred)
    rmse = root_mean_squared_error(y_test, y_pred)
    
    # Set title and labels
    plt.title(f'{model_name}\nR² = {r2:.3f}, RMSE = {rmse:.3f}', fontsize=12, color='lightgray')
    plt.xlabel('Actual Popularity', color='lightgray')
    plt.ylabel('Predicted Popularity', color='lightgray')
    
    # Set tick colors
    plt.tick_params(colors='lightgray')
    
    # Add grid
    plt.grid(True, linestyle='--', alpha=0.7)
    
    # Set equal limits for better comparison
    plt.xlim(0, 100)
    plt.ylim(0, 100)
    
    # Adjust layout
    plt.tight_layout()
    
    # Save the figure
    plt.savefig(f"images/predicted_vs_actual_{model_name.replace(' ', '_').lower()}.png", 
                dpi=300, bbox_inches="tight", transparent=True)

# Show a message instead of displaying all plots at once
print(f"Created and saved {len(models)} individual model plots to images/ directory")


## KFold Cross Validation

Because the Random Forest Regressor performed the best, we will use it for KFold Cross Validation.

In [ ]:
# Assuming 'df' is already loaded and preprocessed
target = "popularity"
features = df.columns.difference(["popularity"])


# Preprocessing function (No scaling needed for Random Forest)
def preprocess_data(df, features, target):
    X = df[features]
    y = df[target]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)
    return X_train, X_test, y_train, y_test


# Nested Cross-Validation function with GridSearchCV
def nested_cv(model, param_grid, X, y, k_outer=5, k_inner=3):
    outer_kf = KFold(n_splits=k_outer, shuffle=True, random_state=42)
    outer_mses = []

    # Store all parameter combinations and their corresponding MSE for visualization
    param_combinations = []
    mse_values = []

    # Outer loop for cross-validation with tqdm progress bar
    for train_index, test_index in tqdm(outer_kf.split(X), total=k_outer, desc="Outer loop"):
        X_train_outer, X_test_outer = X.iloc[train_index], X.iloc[test_index]
        y_train_outer, y_test_outer = y.iloc[train_index], y.iloc[test_index]

        # Inner loop for hyperparameter tuning using GridSearchCV
        inner_kf = KFold(n_splits=k_inner, shuffle=True, random_state=42)
        grid_search = GridSearchCV(
            model, param_grid, cv=inner_kf, scoring="neg_mean_squared_error", verbose=3
        )
        grid_search.fit(X_train_outer, y_train_outer)

        # Store the grid search results
        param_combinations.extend(grid_search.cv_results_["params"])
        mse_values.extend(grid_search.cv_results_["mean_test_score"])

        # Get the best model
        best_model = grid_search.best_estimator_

        # Predictions on the outer fold's test set
        y_pred_outer = best_model.predict(X_test_outer)
        outer_mses.append(mean_squared_error(y_test_outer, y_pred_outer))

    return np.mean(outer_mses), param_combinations, mse_values


# Parameter grid for RandomForest
param_grid = {
    "n_estimators": [100, 200, 300],  # Number of trees
    "max_depth": [40, 50, 60],  # Depth of trees
    "min_samples_split": [2, 5, 10],  # Minimum samples to split
    "n_jobs": [-1],  # Use all processors
    "max_features": ["sqrt"],  # Number of features to consider for the best split
}

# Preprocess the data
X_train, X_test, y_train, y_test = preprocess_data(df, features, target)

# Perform Nested Cross-Validation for RandomForest
nested_mse, param_combinations, mse_values = nested_cv(
    RandomForestRegressor(), param_grid, X_train, y_train
)

# Output the best parameters and the nested cross-validation MSE
print(f"Nested CV Mean MSE: {nested_mse}")

# Convert the results into a DataFrame for easier plotting
results_df = pd.DataFrame(param_combinations)
results_df["mse"] = mse_values

In [ ]:
# # Plotting
# plt.figure(figsize=(12, 8))

# # Plot MSE vs. n_estimators
# plt.subplot(2, 2, 1)
# for max_depth in param_grid["max_depth"]:
#     for min_samples_split in param_grid["min_samples_split"]:
#         subset = results_df[
#             (results_df["max_depth"] == max_depth)
#             & (results_df["min_samples_split"] == min_samples_split)
#         ]
#         plt.plot(
#             subset["n_estimators"],
#             subset["mse"],
#             label=f"max_depth={max_depth}, min_samples_split={min_samples_split}",
#         )

# plt.title("MSE vs. Number of Estimators")
# plt.xlabel("Number of Estimators")
# plt.ylabel("Mean Squared Error")
# plt.legend()

# # Plot MSE vs. max_depth
# plt.subplot(2, 2, 2)
# for n_estimators in param_grid["n_estimators"]:
#     for min_samples_split in param_grid["min_samples_split"]:
#         subset = results_df[
#             (results_df["n_estimators"] == n_estimators)
#             & (results_df["min_samples_split"] == min_samples_split)
#         ]
#         plt.plot(
#             subset["max_depth"],
#             subset["mse"],
#             label=f"n_estimators={n_estimators}, min_samples_split={min_samples_split}",
#         )

# plt.title("MSE vs. Max Depth")
# plt.xlabel("Max Depth")
# plt.ylabel("Mean Squared Error")
# plt.legend()

# # Plot MSE vs. min_samples_split
# plt.subplot(2, 2, 3)
# for n_estimators in param_grid["n_estimators"]:
#     for max_depth in param_grid["max_depth"]:
#         subset = results_df[
#             (results_df["n_estimators"] == n_estimators) & (results_df["max_depth"] == max_depth)
#         ]
#         plt.plot(
#             subset["min_samples_split"],
#             subset["mse"],
#             label=f"n_estimators={n_estimators}, max_depth={max_depth}",
#         )

# plt.title("MSE vs. Min Samples Split")
# plt.xlabel("Min Samples Split")
# plt.ylabel("Mean Squared Error")
# plt.legend()

# plt.tight_layout()
# plt.show()

In [ ]:
# # Plotting
# plt.figure(figsize=(12, 8))

# results_grouped = results_df.groupby(["n_estimators", "max_depth", "min_samples_split"])["mse"].agg("mean").reset_index()

# # Plot MSE vs. n_estimators
# plt.subplot(3, 1, 1)
# for max_depth in param_grid['max_depth']:
#     for min_samples_split in param_grid['min_samples_split']:
#         subset = results_grouped[
#             (results_grouped['max_depth'] == max_depth) &
#             (results_grouped['min_samples_split'] == min_samples_split)
#         ]
#         plt.scatter(subset['n_estimators'], subset['mse'], label=f"max_depth={max_depth}, min_samples_split={min_samples_split}")

# plt.title("MSE vs. Number of Estimators")
# plt.xlabel("Number of Estimators")
# plt.ylabel("Mean Squared Error")
# plt.legend()

# # Plot MSE vs. max_depth
# plt.subplot(3, 1, 2)
# for n_estimators in param_grid['n_estimators']:
#     for min_samples_split in param_grid['min_samples_split']:
#         subset = results_grouped[
#             (results_grouped['n_estimators'] == n_estimators) &
#             (results_grouped['min_samples_split'] == min_samples_split)
#         ]
#         plt.plot(subset['max_depth'], subset['mse'], label=f"n_estimators={n_estimators}, min_samples_split={min_samples_split}")

# plt.title("MSE vs. Max Depth")
# plt.xlabel("Max Depth")
# plt.ylabel("Mean Squared Error")
# plt.legend()

# # Plot MSE vs. min_samples_split
# plt.subplot(3, 1, 3)
# for n_estimators in param_grid['n_estimators']:
#     for max_depth in param_grid['max_depth']:
#         subset = results_grouped[
#             (results_grouped['n_estimators'] == n_estimators) &
#             (results_grouped['max_depth'] == max_depth)
#         ]
#         plt.plot(subset['min_samples_split'], subset['mse'], label=f"n_estimators={n_estimators}, max_depth={max_depth}")

# plt.title("MSE vs. Min Samples Split")
# plt.xlabel("Min Samples Split")
# plt.ylabel("Mean Squared Error")
# plt.tight_layout()
# plt.legend()
# plt.show()
